<a href="https://colab.research.google.com/github/tillns/CIL_project/blob/master/Code/ownGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
use_progressive_variant = True

# TODO:
# 2) a progress bar would be kinda nice as well
# 3) tweak params, of course
# 4) think about batch norm/instance norm
# 
# 6) name files based on parameters and architecture, s.t. there are individual ones for different kinds of training
# 7) Adding noise to real input images before feeding them into the dis might help https://github.com/soumith/ganhacks/issues/14
# 8) save model every few minutes maybe
# 9) progressive growing proposed by Sven: https://github.com/tkarras/progressive_growing_of_gans --- Code at https://github.com/tkarras/progressive_growing_of_gans/blob/master/networks.py
#    not yet implemented use_wscale? weight decay 0.999 I think
#    they seem to generally use float 32, maybe adjust code to that
import os, time, itertools, pickle
import numpy as np
import tensorflow as tf
from PIL import Image
import math

# for now, only those images with label 1 are kept
# consider including also the fake ones

original_image_size = 1000
# e.g. for training on 64x64, the original images may be downsampled to 250x250
# and during training. random 64x64 (appr. 1/4 = 1/patch_divide_factor) patches
# are drawn from these images
image_channels = 1
home_dir = os.path.expanduser("~")
image_directory = os.path.join(home_dir, "dataset/cil-cosmology-2018/cosmology_aux_data_170429/labeled")
label_path = os.path.join(home_dir, "dataset/cil-cosmology-2018/cosmology_aux_data_170429/labeled.csv")
print(label_path)

convs = []
lrelus = []
resblocks = []
upsamples = []
tf.reset_default_graph()
tf.InteractiveSession().close()

# params to tweak
gen_num_upsample = 2
gen_res = pow(2, gen_num_upsample)
gen_num_resblocks = 0
gen_num_normal_convs = 0
dis_num_downsample = 2
dis_res = pow(2, dis_num_downsample)
dis_num_normal_convs = 0
dis_num_resblocks = 0
# my guess is that b&w star pics won't need many feature maps, bc the only 
# features are white dots and black background
gen_start_features = 32
dis_start_features = 16
normal_conv_kernel_dim = 3
normal_kernel = (normal_conv_kernel_dim, normal_conv_kernel_dim)
downsample_conv_kernel_dim = 4
lrelu_factor = 0.2
batch_size = 32
lr = 0.0002
train_epoch = 5
GAN_loss_type = 'LSGAN'
# defined in load_dataset
dataset_length = 0
num_iterations = 0
downsample_size = 0

# some params for progressive growing GAN
max_features = 64
min_features = 2
start_res = 4
# think about difference to 1000 (possibility is a conv that reduces 128 to 125,
# which would require valid paddding with either a kernel size of 4x4 or 
# removing one line to achieve a 127x127 spatial res. Not sure which would be worse ^^)
end_res = original_image_size  
num_res_change = 9
num_const_feature_layers = 4
mbstd_group_size = 4
# cross faded to and from rgb layers (first in gen, last in dis)
to_rgb = []
gen_convs = []
gen_dense = []
from_rgb = []
dis_convs = []
dis_dense = []
dis_pool = tf.keras.layers.AveragePooling2D()
combined_res_change_and_conv = False  # can't be true yet


def load_dataset(image_size=64, patch_divide_factor=4):
  factor_to_downsample = max(1024//image_size//patch_divide_factor, 1)
  global downsample_size
  downsample_size = original_image_size//factor_to_downsample  
  print("Downsampling input images to: {}x{}".format(downsample_size, downsample_size))
  try:
    f=open(label_path,'r')
    label_list = []
    for line in f:
      if not "Id,Actual" in line:
        label_list.append(line.split(","))
    label_list = sorted(label_list)
    labels = np.zeros(len(label_list))
    for ind, label in enumerate(label_list):
      labels[ind] = label[1]

    imgs = np.empty((0, downsample_size, downsample_size, image_channels)) 
    img_list = []
    for filename in os.listdir(image_directory):
      if filename.endswith(".png") and not filename.startswith("._"):
         img_list.append(filename)


    img_list = sorted(img_list)
    for ind, filename in enumerate(img_list):
      if labels[ind] == 1:
        img = Image.open(os.path.join(image_directory, filename)).resize((downsample_size, downsample_size))
        imgs = np.append(imgs, np.array(img).reshape((1, downsample_size, downsample_size, image_channels)), axis=0)
    # imgs = (imgs - 0.5) / 0.5  # if they go from 0 to 1, use this to make them go from -1 to 1


  except FileNotFoundError:
    print("Dataset not found. Using dummy dataset")
    labels = np.ones(1000)
    imgs = np.random.normal(0, 1, (1000, downsample_size, downsample_size, image_channels))
  

  global num_iterations
  global dataset_length
  dataset_length = int(imgs.shape[0])
  num_iterations = dataset_length // batch_size
  # this could get useful if we include the fake images from the dataset with label 0 
  # labels_wide = np.zeros((dataset_length, image_size//dis_res, image_size//dis_res, 1))
  # for ind in range(dataset_length):
  #  labels_wide[ind, :, :, :] = labels[ind]
  return imgs

  
def build_progr_gen_layers(transp=False):
  with tf.variable_scope('generator'):
    features = max_features
    # first special block
    gen_dense.append(tf.keras.layers.Dense(features*start_res*start_res))
    gen_convs.append(tf.keras.layers.Conv2D(features, normal_kernel, padding='same'))
    to_rgb.append(tf.keras.layers.Conv2D(image_channels, kernel_size=(1, 1), padding='same'))

    # rest normal blocks
    for num_upsample in range(num_res_change):
      if num_upsample >= num_const_feature_layers-1: 
        features = features//2

      if transp:
        # Put transposed conv here. Should be more efficient but could also produce checkerboards, we'll see
        pass
      else:
        gen_convs.append(tf.keras.layers.Conv2D(features, normal_kernel, padding='same'))
      if start_res*pow(2, num_upsample) == 128:
        # this should decrease the spatial resolution from 128 to 125 to produce 1000x1000 images in the end
        gen_convs.append(tf.keras.layers.Conv2D(features, kernel_size=(4, 4), padding='valid'))
      else:
        gen_convs.append(tf.keras.layers.Conv2D(features, normal_kernel, padding='same'))
      # every block is followed by a to_rgb conv that are faded into one another
      to_rgb.append(tf.keras.layers.Conv2D(image_channels, kernel_size=(1, 1), padding='same'))
      
def build_progr_dis_layers(strided_conv=False):
  with tf.variable_scope('discriminator'):
    features = min_features

    # downsampling convs (reduce resolution, increase features)
    for num_downsample in range(num_res_change):
      # this layer has to be cross faded for res changes
      from_rgb.append(tf.keras.layers.Conv2D(features, kernel_size=(1, 1), padding='same'))
      print(from_rgb[-1].get_config())

      if end_res//pow(2, num_downsample) == 125:
        # padding: 125x125 -> 131x131, then valid padding conv 4x4: 131x131 -> 128x128. CHECK
        dis_convs.append(tf.keras.layers.Conv2D(features, kernel_size=(4, 4), padding='valid'))
        print(dis_convs[-1].get_config())
      else:
        dis_convs.append(tf.keras.layers.Conv2D(features, normal_kernel, padding='same'))
        print(dis_convs[-1].get_config())
      if num_downsample <= num_res_change-num_const_feature_layers-1:
        features=features*2
      if strided_conv:
        # add strided conv
        pass
      else:
        dis_convs.append(tf.keras.layers.Conv2D(features, normal_kernel, padding='same'))
        print(dis_convs[-1].get_config())

    from_rgb.append(tf.keras.layers.Conv2D(features, kernel_size=(1, 1), padding='same'))
    print(from_rgb[-1].get_config())
    dis_convs.append(tf.keras.layers.Conv2D(features, normal_kernel, strides=(1, 1), padding='same'))
    print(dis_convs[-1].get_config())
    dis_dense.append(tf.keras.layers.Dense(features))
    print(dis_dense[-1].get_config())
    dis_dense.append(tf.keras.layers.Dense(1))
    print(dis_dense[-1].get_config())


/root/dataset/cil-cosmology-2018/cosmology_aux_data_170429/labeled.csv


In [0]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
from random import randint

def lrelu(x, th=0.2):
    return tf.maximum(th * x, x)
  
def pixel_norm(x, epsilon=1e-8):
  return x * tf.rsqrt(tf.reduce_mean(tf.square(x), axis=1, keepdims=True) + epsilon)


# nearest neighbor upscaling. copied from progressive GAN paper; adjusted bc dimensions were ordered differently.
def upscale2d(x, factor=2):
    assert isinstance(factor, int) and factor >= 1
    if factor == 1: return x
    with tf.variable_scope('Upscale2D'):
        s = x.shape
        x = tf.reshape(x, [-1, s[1], 1, s[2], 1, s[3]])
        x = tf.tile(x, [1, 1, factor, 1, factor, 1])
        x = tf.reshape(x, [-1, s[1] * factor, s[2] * factor, s[3]])
        return x

# Minibatch standard deviation.

def minibatch_stddev_layer(x, group_size=4):
    with tf.variable_scope('MinibatchStddev'):
        #group_size = tf.minimum(group_size, tf.shape(x)[0])     # Minibatch must be divisible by (or smaller than) group_size.
        s = x.get_shape().as_list()                                            # [NCHW]  Input shape.
        print(s)
        group_size = min(group_size, batch_size)
        y = tf.reshape(x, (group_size, -1, s[1], s[2], s[3]))   # [GMCHW] Split minibatch into M groups of size G.
        y = tf.cast(y, tf.float32)                              # [GMCHW] Cast to FP32.
        y -= tf.reduce_mean(y, axis=0, keepdims=True)           # [GMCHW] Subtract mean over group.
        y = tf.reduce_mean(tf.square(y), axis=0)                # [MCHW]  Calc variance over group.
        y = tf.sqrt(y + 1e-8)                                   # [MCHW]  Calc stddev over group.
        y = tf.reduce_mean(y, axis=[1,2,3], keepdims=True)      # [M111]  Take average over fmaps and pixels.
        y = tf.cast(y, x.dtype)                                 # [M111]  Cast back to original data type.
        y = tf.tile(y, [group_size, s[1], s[2], 1])             # [N1HW]  Replicate over group and pixels.
        return tf.concat([x, y], axis=3)                        # [NCHW]  Append as new fmap.

      
# in the progression paper, they apply the activation (lrelu) BEFORE the pixel normalization
def activated_norm(x, lrelu_factor=0.2, isTrain=True, normalization='pixel'):
  if normalization == 'pixel':
    return pixel_norm(lrelu(x, lrelu_factor))
  elif normalization == 'batch':
    return lrelu(tf.layers.batch_normalization(x, training=isTrain), lrelu_factor)
  else:
    return lrelu(x, lrelu_factor)

  
  

# number of input features MUST equal number of output features!
def get_resblock(x, features=256, kernel=[3, 3], strides=(1, 1), padding='same', lrelu_factor=0.2, isTrain=True):
    convs.append(tf.layers.conv2d(x, features, kernel, strides, padding))
    lrelus.append(lrelu(tf.layers.batch_normalization(convs[-1], training=isTrain), lrelu_factor))
    convs.append(tf.layers.conv2d(lrelus[-1], features, kernel, strides, padding))
    lrelus.append(lrelu(tf.layers.batch_normalization(convs[-1], training=isTrain), lrelu_factor))
    return lrelus[-1]+x;
  
  
# gen and dis after growing GANs paper. In the paper, they describe their up-/downsampling
# methods to be separate from the conv while in the code, they also explore the combination
# with transposed/strided convolution. Maybe add this here, it's more efficient.  

# TODO: The rough architecture should be right now. Run and fix all errors.
# Also, there is no implementation for the real progression part yet. For this, 
# some layers would probably have to load the saved weights from the training with 
# fewer layers. Not exactly sure how to do it yet.
def generator_progr(x, isTrain=True, reuse=False, transp=False, num_upsample_blocks=0, 
                    cross_fade_alpha=1): # 1 for no cross fading. 
  with tf.variable_scope('generator', reuse=reuse):
    # first conv block
    x = tf.reshape(x, (-1, max_features))  # this assumes the input to have 1x1 spatial res
    x = gen_dense[0](x)
    x = tf.reshape(x, (-1, start_res, start_res, max_features))
    x = activated_norm(x, lrelu_factor, isTrain, normalization='pixel')
    x = gen_convs[0](x)
    x = activated_norm(x, lrelu_factor, isTrain, normalization='pixel')
    alpha = tf.reshape(cross_fade_alpha, (1, 1, 1, 1))  
    if num_upsample_blocks == 0 or cross_fade_alpha == 1:
      add_x = 0
    for num_upsample in range(num_upsample_blocks):

      if not transp:
        x = upscale2d(x)
      if num_upsample == num_upsample_blocks-1 and cross_fade_alpha != 1:  
        add_x = (1-alpha)*to_rgb[num_upsample_blocks-1](x)  # might have to replace it if transp conv is used bc the resolution would be wrong, I think

      x = gen_convs[2*num_upsample+1](x)
      x = activated_norm(x, lrelu_factor, isTrain, normalization='pixel')
      x = gen_convs[2*num_upsample+2](x)
      x = activated_norm(x, lrelu_factor, isTrain, normalization='pixel')

    # apparently, scalar multiplication is problematic for the dynamic shape guess of tf;
    # I don't know why it works for add_x tho

    x = alpha*to_rgb[num_upsample_blocks](x) + add_x

    return x

    
# D(x)
def discriminator_progr(x, isTrain=True, reuse=False, strided_conv=False, num_downsample_blocks=0,
                        cross_fade_alpha=1):
  with tf.variable_scope('discriminator'):
    # this layer has to be cross faded for res changes
    add_x = x
    alpha = tf.reshape(cross_fade_alpha, (1, 1, 1, 1))
    x = alpha*from_rgb[-1-num_downsample_blocks](x)
    x = activated_norm(x, lrelu_factor, isTrain, normalization='None')

    # downsampling convs (reduce resolution, increase features)
    for num_downsample in range(num_downsample_blocks):
      if end_res//pow(2, num_downsample) == 125:
        # padding: 125x125 -> 131x131, then valid padding conv 4x4: 131x131 -> 128x128. CHECK
        x = tf.pad(x, tf.constant([[0, 0], [3, 3], [3, 3], [0, 0]])) 
      x = dis_convs[-3-2*num_downsample](x)
      x = activated_norm(x, lrelu_factor, isTrain, normalization='None')
      x = dis_convs[-2-2*num_downsample](x)
      x = activated_norm(x, lrelu_factor, isTrain, normalization='None')
      if not strided_conv:
        x = dis_pool(x)
        if num_downsample==0 and cross_fade_alpha!=1:
          add_x = dis_pool(add_x)
          x += (1-alpha)*from_rgb[-2-num_downsample_blocks](add_x)

    if mbstd_group_size > 1:
      x = minibatch_stddev_layer(x, mbstd_group_size)
    x = dis_convs[-1](x)
    x = tf.reshape(x, (-1, max_features*start_res*start_res))  # assumes input to be features x start_res x start_res
    x = dis_dense[0](x)
    x = activated_norm(x, lrelu_factor, isTrain, normalization='None')
    x = dis_dense[1](x)

    return x
  
      
    
# G(z)
def generator(x, isTrain=True, reuse=False):
    with tf.variable_scope('generator', reuse=reuse):
      
      features = gen_start_features
      # resblocks (num features kept constant)
      for num_resblock in range(gen_num_resblocks):
          if num_resblock == 0:
            input_im = x
          else:
            input_im = resblocks[-1]
          resblocks.append(get_resblock(input_im, features=features, kernel = [normal_conv_kernel_dim, normal_conv_kernel_dim], lrelu_factor=lrelu_factor, isTrain=isTrain))

      # upsample + conv (num features reduced)
      for num_upsample in range(gen_num_upsample):
        if num_upsample == 0:
          if gen_num_resblocks == 0:
            input_im = x
          else:
            input_im = resblocks[-1]
        else:
          input_im = lrelus[-1]
        print("gen 1 shape: {}".format(input_im.shape))
        upsamples.append(upscale2d(input_im))
        convs.append(tf.layers.conv2d(upsamples[-1], features, [normal_conv_kernel_dim, normal_conv_kernel_dim], strides=(1, 1), padding='same'))
        print("Len convs: {}".format(len(convs)))
        # maybe change to instance normalization (tf.contrib.layers.instance_norm(input, trainable)) 
        # this does not exist in tf 1.3 yet and I haven't gotten it to work on 1.13 either
        # error probably lies in trainable, haven't understood that yet
        lrelus.append(lrelu(tf.layers.batch_normalization(convs[-1], training=isTrain), lrelu_factor))
        features = features//2

      # output layer
      if gen_num_upsample == 0:
        if gen_num_resblocks == 0:
          input_im = x
        else:
          input_im = resblocks[-1]
      else:
        input_im = lrelus[-1]
      print("gen 2 shape: {}".format(input_im.shape))
      conv5 = tf.layers.conv2d(input_im, image_channels, [normal_conv_kernel_dim, normal_conv_kernel_dim], strides=(1, 1), padding='same')
      # never normalize output
      o = tf.nn.tanh(conv5)

      return o

# D(x)
def discriminator(x, isTrain=True, reuse=False):
    with tf.variable_scope('discriminator', reuse=reuse):
      
      features = dis_start_features
      # downsampling convs (reduce resolution, increse features)
      for num_downsample in range(dis_num_downsample):
        if num_downsample == 0:
            input_im = x
        else:
          input_im = lrelus[-1]
        convs.append(tf.layers.conv2d(input_im, features, [downsample_conv_kernel_dim, downsample_conv_kernel_dim], strides=(2, 2), padding='same'))
        # maybe change to instance normalization
        lrelus.append(lrelu(tf.layers.batch_normalization(convs[-1], training=isTrain), lrelu_factor))
        features = features*2
        
      # resblocks (features kept constant)  
      for num_resblock in range(dis_num_resblocks):
        if num_resblock == 0:
          if dis_num_downsample == 0:
            input_im = x
          else:
            input_im = lrelus[-1]
            features=features//2
        else:
          input_im = resblocks[-1]
        resblocks.append(get_resblock(input_im, features=features, kernel = [normal_conv_kernel_dim, normal_conv_kernel_dim], lrelu_factor=lrelu_factor, isTrain=isTrain))

      # output layer without fc, s.t. local patches are rated and generator is
      # forced to be cinsistent everywhere (PatchGAN). Could be changed tho...
      if dis_num_resblocks == 0:
        if dis_num_downsample == 0:
          input_im = x
        else:
          input_im = lrelus[-1]
      else:
        input_im = resblocks[-1]
      conv6 = tf.layers.conv2d(input_im, 1, [normal_conv_kernel_dim, normal_conv_kernel_dim], strides=(1, 1), padding='same')
      #fc = tf.layers.dense(conv6, 32*32, activation='sigmoid')
      o = tf.nn.sigmoid(conv6)
      print("D output shape: {}".format(o.shape))
      return o
    
def D(x, isTrain=True, reuse=False, strided_conv=False, num_downsample_blocks=0,
                        cross_fade_alpha=1):
  if use_progressive_variant:
    return discriminator_progr(x, isTrain, reuse, strided_conv, 
                               num_downsample_blocks, cross_fade_alpha)
  else:
    return discriminator(x, isTrain, reuse)
  
def G(x, isTrain=True, reuse=False, transp=False, num_upsample_blocks=0,
                        cross_fade_alpha=1):
  if use_progressive_variant:
    return generator_progr(x, isTrain, reuse, transp, num_upsample_blocks,
                           cross_fade_alpha)
  else:
    return generator(x, isTrain, reuse)

This is the WGAN-GP loss for the generator and discriminator, copied from the growing GAN paper. 

In [0]:
# linearly interpolate between a and b
def lerp(a, b, t):
    with tf.name_scope('Lerp'):
        return a + (b - a) * t
      
def exp2(x):
    with tf.name_scope('Exp2'):
        return tf.exp(x * np.float32(np.log(2.0)))
      
def apply_loss_scaling(value, use_loss_scaling=False):
    assert is_tf_expression(value)
    if not use_loss_scaling:
        return value
    return value * exp2(value)

# Undo the effect of dynamic loss scaling for the given expression.
def undo_loss_scaling(value):
    assert is_tf_expression(value, use_loss_scaling=False)
    if not use_loss_scaling:
        return value
    return value * exp2(-value)
      
def autosummary(name, value):
    id = name.replace('/', '_')
    if is_tf_expression(value):
        with tf.name_scope('summary_' + id), tf.device(value.device):
            update_op = _create_autosummary_var(name, value)
            with tf.control_dependencies([update_op]):
                return tf.identity(value)
    else: # python scalar or numpy array
        if name not in _autosummary_immediate:
            with absolute_name_scope('Autosummary/' + id), tf.device(None), tf.control_dependencies(None):
                update_value = tf.placeholder(tf.float32)
                update_op = _create_autosummary_var(name, update_value)
                _autosummary_immediate[name] = update_op, update_value
        update_op, update_value = _autosummary_immediate[name]
        run(update_op, {update_value: np.float32(value)})
        return value
      
def fp32(*values):
    if len(values) == 1 and isinstance(values[0], tuple):
        values = values[0]
    values = tuple(tf.cast(v, tf.float32) for v in values)
    return values if len(values) >= 2 else values[0]

# wgangp losses. I crossed out the label penalty terms. I think those were introduced in
# https://arxiv.org/pdf/1610.09585.pdf the ac-gan paper. For multi-class generation,
# this term is useful, so it's probably not necessary in this work
def G_wgangp(labels, fake_scores_out,
    cond_weight = 1.0): # Weight of the conditioning term.

    #latents = tf.random_normal([batch_size][image_size])  
    #labels = training_set.get_random_labels_tf(minibatch_size)
    #fake_images_out = G(latents, labels, is_training=True)
    #fake_scores_out, fake_labels_out = fp32(D(fake_images_out, is_training=True))
    loss = -fake_scores_out

    # crossed this out
    #if D.output_shapes[1][1] > 0: 
    #with tf.name_scope('LabelPenalty'):
    #    label_penalty_fakes = tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=fake_labels_out)
    #loss += label_penalty_fakes * cond_weight
    return loss


def D_wgangp(minibatch_size, reals, fake_images_out, real_scores_out, fake_scores_out,
    wgan_lambda     = 10.0,     # Weight for the gradient penalty term.
    wgan_epsilon    = 0.001,    # Weight for the epsilon term, \epsilon_{drift}.
    wgan_target     = 1.0,      # Target value for gradient magnitudes.
    cond_weight     = 1.0):     # Weight of the conditioning terms.

    #latents = tf.random_normal([minibatch_size] + G.input_shapes[0][1:])
    #fake_images_out = G.get_output_for(latents, labels, is_training=True)
    #real_scores_out, real_labels_out = fp32(D.get_output_for(reals, is_training=True))
    #fake_scores_out, fake_labels_out = fp32(D.get_output_for(fake_images_out, is_training=True))
    real_scores_out = tfutil.autosummary('Loss/real_scores', real_scores_out)
    fake_scores_out = tfutil.autosummary('Loss/fake_scores', fake_scores_out)
    loss = fake_scores_out - real_scores_out  # this is the normale WGAN loss

    with tf.name_scope('GradientPenalty'):
        mixing_factors = tf.random_uniform([minibatch_size, 1, 1, 1], 0.0, 1.0, dtype=fake_images_out.dtype)
        mixed_images_out = tfutil.lerp(tf.cast(reals, fake_images_out.dtype), fake_images_out, mixing_factors)
        mixed_scores_out, mixed_labels_out = fp32(D(mixed_images_out, is_training=True))
        mixed_scores_out = tfutil.autosummary('Loss/mixed_scores', mixed_scores_out)
        mixed_loss = apply_loss_scaling(tf.reduce_sum(mixed_scores_out))
        mixed_grads = undo_loss_scaling(fp32(tf.gradients(mixed_loss, [mixed_images_out])[0]))
        mixed_norms = tf.sqrt(tf.reduce_sum(tf.square(mixed_grads), axis=[1,2,3]))
        mixed_norms = tfutil.autosummary('Loss/mixed_norms', mixed_norms)
        gradient_penalty = tf.square(mixed_norms - wgan_target)
    loss += gradient_penalty * (wgan_lambda / (wgan_target**2))  # this is the gradient penalty term

    with tf.name_scope('EpsilonPenalty'):
        epsilon_penalty = tfutil.autosummary('Loss/epsilon_penalty', tf.square(real_scores_out))
    loss += epsilon_penalty * wgan_epsilon  # this is the small additional loss mentioned to avoid drifts from zero

    #if D.output_shapes[1][1] > 0:
    #with tf.name_scope('LabelPenalty'):
    #    label_penalty_reals = tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=real_labels_out)
    #    label_penalty_fakes = tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=fake_labels_out)
    #    label_penalty_reals = tfutil.autosummary('Loss/label_penalty_reals', label_penalty_reals)
    #    label_penalty_fakes = tfutil.autosummary('Loss/label_penalty_fakes', label_penalty_fakes)
    #loss += (label_penalty_reals + label_penalty_fakes) * cond_weight
    return loss


In [0]:
def show_result(num_epoch, sess, G_z, z, isTrain, cross_fade_alpha, alpha_=1, show = False, save = False, path = 'result.png', image_size=64):
  size_figure_grid = 5
  if use_progressive_variant:
    spatial = 1
    num_features = max_features
  else:
    spatial = image_size//gen_res
    num_features = gen_start_features
  fixed_z_ = np.random.normal(0, 1, (size_figure_grid*size_figure_grid, spatial, spatial, num_features))
  test_images = sess.run(G_z, {z: fixed_z_, isTrain: False, cross_fade_alpha: alpha_})

  fig_size = image_size*size_figure_grid*1.3/100  # appr. formula s.t. image resolutions aren't reduced
  fig, ax = plt.subplots(size_figure_grid, size_figure_grid, figsize=(fig_size, fig_size))
  for i, j in itertools.product(range(size_figure_grid), range(size_figure_grid)):
      ax[i, j].get_xaxis().set_visible(False)
      ax[i, j].get_yaxis().set_visible(False)

  for k in range(size_figure_grid*size_figure_grid):
      i = k // size_figure_grid
      j = k % size_figure_grid
      ax[i, j].cla()
      ax[i, j].imshow(np.reshape(test_images[k], (image_size, image_size)), cmap='gray')

  label = 'Epoch {0}'.format(num_epoch)
  fig.text(0.5, 0.04, label, ha='center')

  if save:
      plt.savefig(path)

  if show:
      plt.show()
  else:
      plt.close()

# plots graph with dis and gen loss 
def show_train_hist(hist, show = False, save = False, path = 'Train_hist.png'):
    x = range(len(hist['D_losses']))

    y1 = hist['D_losses']
    y2 = hist['G_losses']

    plt.plot(x, y1, label='D_loss')
    plt.plot(x, y2, label='G_loss')

    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()

    if save:
        plt.savefig(path)

    if show:
        plt.show()
    else:
        plt.close()


#with tf.device('/cpu:0'):
def train(imgs, num_res_change_layers=0, cross_fade=False, device='/device:GPU:0', image_size=64):
  with tf.device(device): 
    global gen_res
    global gen_start_features
    if use_progressive_variant:
      gen_res = image_size
      gen_start_features = max_features
    # variables : input
    x = tf.placeholder(tf.float32, shape=(None, image_size, image_size, image_channels))
    z = tf.placeholder(tf.float32, shape=(None, image_size//gen_res, image_size//gen_res, gen_start_features))
    cross_fade_alpha = tf.placeholder(dtype=tf.float32)
    isTrain = tf.placeholder(dtype=tf.bool)
    
    

    # networks : generator
    G_z = G(z, isTrain, transp=combined_res_change_and_conv, num_upsample_blocks=num_res_change_layers, cross_fade_alpha=cross_fade_alpha)

    # networks : discriminator
    D_real_logits = D(x, isTrain, strided_conv=combined_res_change_and_conv, num_downsample_blocks=num_res_change_layers, cross_fade_alpha=cross_fade_alpha)
    D_fake_logits = D(G_z, isTrain, reuse=True, strided_conv=combined_res_change_and_conv, num_downsample_blocks=num_res_change_layers, cross_fade_alpha=cross_fade_alpha)

    # this assumes that the progressive variant has an fc at the end of the dis and the other one does not
    if use_progressive_variant: 
      label_zero = tf.zeros([batch_size, 1])
      label_one = tf.ones([batch_size, 1])
    else:
      label_zero = tf.zeros([batch_size, image_size//dis_res, image_size//dis_res, 1])
      label_one = tf.ones([batch_size, image_size//dis_res, image_size//dis_res, 1])
      
    # loss for each network
    # think about GAN objective
    if GAN_loss_type == 'LSGAN':
      print("Using LSGAN. Might still be wrong.")
      D_loss = tf.reduce_mean(tf.square(D_fake_logits - label_zero)) + tf.reduce_mean(tf.square(D_real_logits - label_one))
      G_loss = tf.reduce_mean(tf.square(D_fake_logits - label_one))
    elif GAN_loss_type == 'WGAN-GP':
      print("Using WGAN-GP without label penalty.")
      G_loss = G_wgangp(label_one, D_fake_logits)
      D_loss = D_wgangp(batch_size, x, G(z), D_real_logits, D_fake_logits)
    else:
      print("Using the GAN objective that was already implemented, not sure which it is")
      D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=label_one))
      D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=label_zero))
      D_loss = D_loss_real + D_loss_fake
      G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=label_one))

    # trainable variables for each network
    T_vars = tf.trainable_variables()
    D_vars = [var for var in T_vars if var.name.startswith('discriminator')]
    G_vars = [var for var in T_vars if var.name.startswith('generator')]

    # optimizer for each network
    # This should probably not be experimented with. ADAM is standard nowadays.
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):      
      D_optimizer = tf.train.AdamOptimizer(lr, beta1=0.5)
      G_optimizer = tf.train.AdamOptimizer(lr, beta1=0.5)

      # access the gradients - HANNES
      D_grads = D_optimizer.compute_gradients(D_loss, var_list=D_vars)
      G_grads = G_optimizer.compute_gradients(G_loss, var_list=G_vars)

      D_optim = D_optimizer.apply_gradients(D_grads)
      G_optim = G_optimizer.apply_gradients(G_grads)


    # create loss summaries for tensorboard - HANNES
    D_loss_summary = tf.summary.scalar(name="D_Loss", tensor=D_loss)

    G_loss_summary = tf.summary.scalar(name="G_loss", tensor=G_loss)

    # create weight summaries for tensorboard - HANNES
    """
    with tf.variable_scope("generator", reuse=True):
      with tf.variable_scope("conv2d", reuse=True): # feature reduction
        g_conv2d_weights_summary = tf.summary.histogram(name="g_conv2d_weights", values=tf.get_variable("kernel"))
      with tf.variable_scope("conv2d_1", reuse=True): # feature reduction
        g_conv2d_1_weights_summary = tf.summary.histogram(name="g_conv2d_1_weights", values=tf.get_variable("kernel"))
      with tf.variable_scope("conv2d_2", reuse=True): # output
        g_conv2d_2_weights_summary = tf.summary.histogram(name="g_conv2d_2_weights", values=tf.get_variable("kernel"))

    with tf.variable_scope("discriminator", reuse=True):
      with tf.variable_scope("conv2d", reuse=True): # feature increase
        d_conv2d_weights_summary = tf.summary.histogram(name="d_conv2d_weights", values=tf.get_variable("kernel"))
      with tf.variable_scope("conv2d_1", reuse=True): # feature increase
        d_conv2d_1_weights_summary = tf.summary.histogram(name="d_conv2d_1_weights", values=tf.get_variable("kernel"))
      with tf.variable_scope("conv2d_2", reuse=True): # output
        d_conv2d_2_weights_summary = tf.summary.histogram(name="d_conv2d_2_weights", values=tf.get_variable("kernel"))
    """
    if cross_fade_alpha != 1:
      for grad in D_grads:
        print(grad)

    # create gradient summaries for tensorboard - HANNES
    for index, grad in enumerate(D_grads):
      tf.summary.histogram("{}_grad".format(D_grads[index][1].name.replace(":", "_")), D_grads[index])
    for index, grad in enumerate(G_grads):
      tf.summary.histogram("{}grad".format(G_grads[index][1].name.replace(":", "_")), G_grads[index])

    # merge summaries - HANNES
    all_summaries = tf.summary.merge_all() 


    # open session and initialize all variables
    sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
    tf.global_variables_initializer().run()


    # visualize graph in tensorboard - HANNES
    fileWriter = tf.summary.FileWriter("./ownGan_logdir", tf.get_default_graph())


    # resize and normalization
    # train_set = dataset#.eval()
    #train_set = (train_set - 0.5) / 0.5  # normalization; range: -1 ~ 1

    # results save folder
    root = 'DCGAN_results/'
    model = 'DCGAN_'
    if not os.path.isdir(root):
        os.mkdir(root)
    if not os.path.isdir(root + 'Fixed_results'):
        os.mkdir(root + 'Fixed_results')

    train_hist = {}
    train_hist['D_losses'] = []
    train_hist['G_losses'] = []
    train_hist['per_epoch_ptimes'] = []
    train_hist['total_ptime'] = []

    # training-loop
    np.random.seed(int(time.time()))
    print('training start!')
    start_time = time.time()
    for epoch in range(train_epoch):
      G_losses = []
      D_losses = []
      epoch_start_time = time.time()
      alpha_ = 1
      #dataset.shuffle(buffer_size=dataset_length)
      #dataset = dataset.batch(batch_size)
      #iterator = dataset.make_one_shot_iterator()

      # TODO: include progress bar
      # to avoid overfitting, the images are shuffled every epoch
      # if fake images are included, shuffle an index list to access the corresponding label along with the image
      np.random.shuffle(imgs)
      for iteration in range(num_iterations):
        #x_ = iterator.get_next()
        #x_ = x_.eval()

        # update discriminator
        x_ = imgs[iteration*batch_size:(iteration+1)*batch_size]
        random_startx = randint(0, downsample_size-image_size)
        random_starty = randint(0, downsample_size-image_size)
        # draw a random patch from the input
        x_ = x_[:, random_startx:random_startx+image_size, random_starty:random_starty+image_size]
        for i in range(2):
          if randint(0, 1):
            x_ = np.flip(x_, axis=i+1)  # randomly flip x- and y-axis


        z_ = np.random.normal(0, 1, (batch_size, image_size//gen_res, image_size//gen_res, gen_start_features))
        if cross_fade:
          alpha_ = (epoch*num_iterations+iteration)/(train_epoch*num_iterations)  # should gradually go from 0 to 1. CHECK
          print("Current alpha: {}".format(alpha_))

        # just do one single sess.run() - HANNES
        summary, loss_d_, _, loss_g_, _ = sess.run([all_summaries, D_loss, D_optim, G_loss, G_optim], {x: x_, z: z_, isTrain: True, cross_fade_alpha: alpha_})
        D_losses.append(loss_d_)
        G_losses.append(loss_g_)

        # save one data point per epoch for tensorboard - HANNES
        if(iteration == dataset_length // batch_size - 1):
          fileWriter.add_summary(summary, epoch)


    #       loss_d_, _ = sess.run([D_loss, D_optim], {x: x_, z: z_, isTrain: True})
    #       D_losses.append(loss_d_)

    #       # update generator
    #       z_ = np.random.normal(0, 1, (batch_size, image_size//gen_res, image_size//gen_res, gen_start_features))
    #       loss_g_, _ = sess.run([G_loss, G_optim], {z: z_, x: x_, isTrain: True})
    #       G_losses.append(loss_g_)

      epoch_end_time = time.time()
      per_epoch_ptime = epoch_end_time - epoch_start_time
      print('[%d/%d] - ptime: %.2f loss_d: %.3f, loss_g: %.3f' % ((epoch + 1), train_epoch, per_epoch_ptime, np.mean(D_losses), np.mean(G_losses)))
      fixed_p = root + 'Fixed_results/' + model + str(epoch + 1) + '.png'
      show_result((epoch + 1), sess, G_z, z, isTrain, cross_fade_alpha, alpha_, save=True, show=False, path=fixed_p, image_size=image_size)
      train_hist['D_losses'].append(np.mean(D_losses))
      train_hist['G_losses'].append(np.mean(G_losses))
      train_hist['per_epoch_ptimes'].append(per_epoch_ptime)

    end_time = time.time()
    total_ptime = end_time - start_time
    train_hist['total_ptime'].append(total_ptime)

    print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(train_hist['per_epoch_ptimes']), train_epoch, total_ptime))
    print("Training finish!... save training results")
    with open(root + model + 'train_hist.pkl', 'wb') as f:
        pickle.dump(train_hist, f)

    show_train_hist(train_hist, save=True, path=root + model + 'train_hist.png')

    # this thing makes a gif out of the results after every epoch, so it's not
    # necessary 
    try:
      import imageio
      images = []
      for e in range(train_epoch):
          img_name = root + 'Fixed_results/' + model + str(e + 1) + '.png'
          images.append(imageio.imread(img_name))
      imageio.mimsave(root + model + 'generation_animation.gif', images, fps=5)
    except Exception as e:
      print("Haven't gotten imageio to work on the cluster, smh. Error message:")
      print(e)


In [5]:
if use_progressive_variant:
  build_progr_gen_layers(combined_res_change_and_conv)
  build_progr_dis_layers(combined_res_change_and_conv)
  current_res = start_res
  for counter in range(num_res_change):

    imgs = load_dataset(current_res)
    if counter == 0:
      train(imgs, counter, False, image_size=current_res)
    else:
      train(imgs, counter, True, image_size=current_res)
      train(imgs, counter, False, image_size=current_res)
      
    current_res = 2*current_res
    if current_res == 128:
      current_res = 125  # little res change s.t. final image size is 1000, not 1024
      
  
else:
  image_size=64
  imgs = load_dataset(image_size=image_size)
  train(imgs, image_size=image_size)
  

{'name': 'conv2d_29', 'trainable': True, 'dtype': None, 'filters': 2, 'kernel_size': (1, 1), 'strides': (1, 1), 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None, 'dtype': 'float32'}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {'dtype': 'float32'}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
{'name': 'conv2d_30', 'trainable': True, 'dtype': None, 'filters': 2, 'kernel_size': (3, 3), 'strides': (1, 1), 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None, 'dtype': 'float32'}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {'dtype': 'float32'}}, 'kernel_regularizer': None, 'bias_regularizer': 

ValueError: ignored

*# Load Dataset*

In [0]:
tf.reset_default_graph()

In [0]:
sess.close()